In [ ]:
import sympy as sp
from scipy.linalg import expm
import numpy as np
import warnings

# --- Helper functions (keep as they are) ---
def coupling_operator_with_phase(i, j, dim, phi):
    op = np.zeros((dim, dim), dtype=complex)
    op[i, j] = np.exp(+1j * phi)
    op[j, i] = np.exp(-1j * phi)
    return op

def pulse_duration_for_fraction(f, Omega):
    theta = np.pi * f
    return theta / Omega if Omega != 0 else 0.0

def unitary(couplings, rabi_freqs, fractions, fixed_phase_flags, dim):
    U_seq = np.eye(dim, dtype=complex)
    for (levels, Omega, frac, fix_pflag) in zip(couplings, rabi_freqs, fractions, fixed_phase_flags):
        i, j = levels
        phi_fixed = fix_pflag * np.pi
        H_op = coupling_operator_with_phase(i, j, dim, phi_fixed)
        H_coupling = 0.5 * Omega * H_op
        t_pulse = pulse_duration_for_fraction(frac, Omega)
        U_pulse = expm(-1j * H_coupling * t_pulse)
        U_seq = U_pulse @ U_seq
    return U_seq

def fix_couplings_and_phases(couplings, fixed_phase_flags):
    new_couplings = []
    new_fixed_phase_flags = []
    for (cpl, phase_flag) in zip(couplings, fixed_phase_flags):
        i, j = cpl
        if i != 0 and j == 0:
            cpl_fixed = (0, i)
            phase_flag_fixed = phase_flag + 1.0
        else:
            cpl_fixed = cpl
            phase_flag_fixed = phase_flag
        new_couplings.append(cpl_fixed)
        new_fixed_phase_flags.append(phase_flag_fixed)
    return new_couplings, new_fixed_phase_flags

# --- Symbolic Givens Rotation ---
def givens_rotation_symbolic(i, j, theta, phi, dim):
    G = sp.eye(dim)
    cos_term = sp.cos(theta / 2)
    sin_term = sp.sin(theta / 2)
    e_neg_iphi = sp.exp(-sp.I * phi)
    e_pos_iphi = sp.conjugate(e_neg_iphi)

    G[i, i] = cos_term
    G[j, j] = cos_term
    G[i, j] = -sp.I * e_pos_iphi * sin_term
    G[j, i] = -sp.I * e_neg_iphi * sin_term
    return G

# --- Main Script for Multi-Pulse nsolve Test ---

# Dimension
dim = 4

# Define sequence parameters
couplings = [(0, 3)]#, (0, 1)]#, (0, 2), (0, 1), (0, 3)]
fractions = [
    0.5
    # 2.0 * np.arcsin(np.sqrt(1/3)) / np.pi,
    # 4/3,
    # 2.0 * np.arcsin(np.sqrt(2/3)) / np.pi + 1,
    # 0.5
]
fixed_phase_flags = [1.5]#, 0.5, 1.5, 0.5, 0.5]
rabi_freqs = [1] * len(couplings)

# Apply fixing (for couplings and phases)
couplings_fixed, fixed_phase_flags_fixed = fix_couplings_and_phases(
    couplings, fixed_phase_flags
)

num_pulses = len(couplings_fixed)

# Create symbolic parameters: theta_0..theta_{num_pulses-1}, phi_0..phi_{num_pulses-1}
theta_syms = sp.symbols(f'theta_0:{num_pulses}', real=True)
phi_syms = sp.symbols(f'phi_0:{num_pulses}', real=True)
params = list(theta_syms) + list(phi_syms)

# Build the symbolic unitary for the full sequence
U_symbolic = sp.eye(dim)
for k, levels in enumerate(couplings_fixed):
    i, j = levels
    theta_k = theta_syms[k]
    phi_k = phi_syms[k]
    G_k = givens_rotation_symbolic(i, j, theta_k, phi_k, dim)
    U_symbolic = G_k @ U_symbolic

# Generate the target U numerically
target_U = unitary(
    couplings_fixed,
    rabi_freqs,
    fractions,
    fixed_phase_flags_fixed,
    dim
)
target_U_sym = sp.Matrix(target_U)

# Setup equations: match real parts of diagonal and imaginary parts of selected off-diagonals
equations = []
# Real(diag) = Real(target diag)
for idx in range(dim):
    equations.append(sp.re(U_symbolic[idx, idx]) - sp.re(target_U_sym[idx, idx]))
# Imag parts of some off-diagonals
off_diag_idxs = [(0,3)]#, (0,1), (0,2), (0,1), (0,3)]
for (i,j) in off_diag_idxs:
    equations.append(sp.im(U_symbolic[i, j]) - sp.im(target_U_sym[i, j]))
# Add one more equation to reach 2*num_pulses = 10
equations.append(sp.im(U_symbolic[1, 2]) - sp.im(target_U_sym[1, 2]))

# Define initial guesses based on expected values
expected_thetas = [f * np.pi for f in fractions]
expected_phis = [pf * np.pi for pf in fixed_phase_flags_fixed]
initial_guesses = [et * 0.1 for et in expected_thetas] + [ep * 0.1 for ep in expected_phis]

# Solve numerically with nsolve
print("Solving for symbolic parameters...")
sol = sp.nsolve(
    equations,
    params,
    initial_guesses,
    # tol=1e-14,
    # maxsteps=50,
    # prec=30     
)

# Display solutions
for k in range(num_pulses):
    print(f"theta_{k} =", solutions[k]/np.pi)
    print(f"phi_{k}   =", solutions[num_pulses + k]/np.pi)


In [20]:
import sympy as sp
from scipy.linalg import expm
import numpy as np
import matplotlib.pyplot as plt

def coupling_operator_with_phase(i, j, dim, phi):
    op = np.zeros((dim, dim), dtype=complex)
    op[i, j] = np.exp(+1j * phi)
    op[j, i] = np.exp(-1j * phi)
    return op

def pulse_duration_for_fraction(f, Omega):
    theta = np.pi * np.array(f)  # Angle is π * f
    return theta / Omega if Omega != 0 else 0.0

def unitary(couplings, rabi_freqs, fractions, fixed_phase_flags, dim):
    U_seq = np.eye(dim, dtype=complex)
    for (levels, Omega, frac, fix_pflag) in zip(couplings, rabi_freqs, fractions, fixed_phase_flags):
        i, j = levels
        phi_fixed = fix_pflag * np.pi
        total_phase = phi_fixed
        H_op = coupling_operator_with_phase(i, j, dim, total_phase)
        H_coupling = 0.5 * Omega * H_op
        t_pulse = pulse_duration_for_fraction(frac, Omega)
        U_pulse = expm(-1j * H_coupling * t_pulse)
        U_seq = U_pulse @ U_seq
    return U_seq

def fix_couplings_and_phases(couplings, fixed_phase_flags):
    new_couplings = []
    new_fixed_phase_flags = []
    for (cpl, phase_flag) in zip(couplings, fixed_phase_flags):
        i, j = cpl
        if i != 0 and j == 0:
            cpl_fixed = (0, i)
            phase_flag_fixed = phase_flag + 1.0
        else:
            cpl_fixed = cpl
            phase_flag_fixed = phase_flag
        new_couplings.append(cpl_fixed)
        new_fixed_phase_flags.append(phase_flag_fixed)
    return new_couplings, new_fixed_phase_flags

def givens_rotation_symbolic(i, j, theta, phi, dim):
    """Symbolic Givens rotation acting on levels i and j in a dim-dimensional space."""
    G = sp.eye(dim)
    cos = sp.cos(theta / 2)
    sin = sp.sin(theta / 2)
    eiphase = sp.exp(-sp.I * phi)

    G_block = sp.Matrix([
        [cos, -sp.I * sp.conjugate(eiphase) * sin],
        [-sp.I * eiphase * sin, cos]
    ])

    rows = [i, j]
    cols = [i, j]
    for r in range(2):
        for c in range(2):
            G[rows[r], cols[c]] = G_block[r, c]
    return G

# Dimension
dim = 4

# Define symbolic parameters
params = []
for k in range(5):
    theta = sp.symbols(f'theta_{k}', real=True)
    phi = sp.symbols(f'phi_{k}', real=True)
    params.extend([theta, phi])

# Define symbolic Givens sequence
sequence = [(0, 3)]

U = sp.eye(dim)
for k, (i, j) in enumerate(sequence):
    theta = params[2 * k]
    phi = params[2 * k + 1]
    G = givens_rotation_symbolic(i, j, theta, phi, dim)
    U = G * U

# Define numerical parameters
couplings = [(0, 3)]
fractions = [0.5]
fixed_phase_flags = [1.5]
rabi_freqs = [1]

# Fix ordering and phase flags
couplings, fixed_phase_flags = fix_couplings_and_phases(couplings, fixed_phase_flags)
print("Couplings and phases:", couplings, fixed_phase_flags)

# Compute numeric target matrix
target = unitary(couplings, rabi_freqs, fractions, fixed_phase_flags, dim)

# Generate symbolic equations
equations = []
for i in range(dim):
    for j in range(dim):
        real_part = sp.re(U[i, j])
        imag_part = sp.im(U[i, j])
        target_real = sp.nsimplify(np.real(target[i, j]), rational=True)
        target_imag = sp.nsimplify(np.imag(target[i, j]), rational=True)
        equations.append(sp.Eq(real_part, target_real))
        equations.append(sp.Eq(imag_part, target_imag))

print(f"Generated {len(equations)} real equations for {len(params)} real unknowns.")

# Solve equations
solutions = sp.solve(equations, params, dict=True)

print("Equations:")
for eq in equations:
    print(eq)

print("\nSolutions:")
for sol in solutions:
    print(sol)


Couplings and phases: [(0, 3)] [1.5]
Generated 32 real equations for 10 real unknowns.
Equations:
Eq(cos(theta_0/2), 176776695296637/250000000000000)
False
True
True
True
True
Eq(sin(phi_0)*sin(theta_0/2), -707106781186547/1000000000000000)
Eq(-sin(theta_0/2)*cos(phi_0), 32473352108831/250000000000000000000000000000)
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
Eq(-sin(phi_0)*sin(theta_0/2), 707106781186547/1000000000000000)
Eq(-sin(theta_0/2)*cos(phi_0), 32473352108831/250000000000000000000000000000)
True
True
True
True
Eq(cos(theta_0/2), 176776695296637/250000000000000)
True

Solutions:


In [25]:
import sympy as sp
from scipy.linalg import expm
import numpy as np
import matplotlib.pyplot as plt
import warnings
import time # To time the solver

# --- Helper functions (keep as they are) ---
def coupling_operator_with_phase(i, j, dim, phi):
    # ... (same as before)
    op = np.zeros((dim, dim), dtype=complex)
    op[i, j] = np.exp(+1j * phi)
    op[j, i] = np.exp(-1j * phi)
    return op

def pulse_duration_for_fraction(f, Omega):
    # ... (same as before)
    theta = np.pi*np.array(f)
    return theta / Omega if Omega != 0 else 0.0

def unitary(couplings, rabi_freqs, fractions, fixed_phase_flags, dim):
    # ... (same as before)
    U_seq = np.eye(dim, dtype=complex)
    for (levels, Omega, frac, fix_pflag) in zip(couplings, rabi_freqs, fractions, fixed_phase_flags):
        i, j = levels
        phi_fixed = fix_pflag * np.pi
        total_phase = phi_fixed
        H_op = coupling_operator_with_phase(i, j, dim, total_phase)
        H_coupling = 0.5 * Omega * H_op
        t_pulse = pulse_duration_for_fraction(frac, Omega)
        U_pulse = expm(-1j * H_coupling * t_pulse)
        U_seq = U_pulse @ U_seq
    return U_seq

def fix_couplings_and_phases(couplings, fixed_phase_flags):
    # ... (same as before)
    new_couplings = []
    new_fixed_phase_flags = []
    for (cpl, phase_flag) in zip(couplings, fixed_phase_flags):
        i, j = cpl
        if i != 0 and j == 0:
            cpl_fixed = (0, i)
            phase_flag_fixed = phase_flag + 1.0
        else:
            cpl_fixed = cpl
            phase_flag_fixed = phase_flag
        new_couplings.append(cpl_fixed)
        new_fixed_phase_flags.append(phase_flag_fixed)
    return new_couplings, new_fixed_phase_flags

# --- Symbolic Givens Rotation ---
def givens_rotation_symbolic(i, j, theta, phi, dim):
    # ... (same as before)
    G = sp.eye(dim)
    cos_term = sp.cos(theta / 2)
    sin_term = sp.sin(theta / 2)
    e_neg_iphi = sp.exp(-sp.I * phi)
    e_pos_iphi = sp.conjugate(e_neg_iphi)

    G[i, i] = cos_term
    G[j, j] = cos_term
    G[i, j] = -sp.I * e_pos_iphi * sin_term
    G[j, i] = -sp.I * e_neg_iphi * sin_term
    return G

# --- Main Script for Full Sequence nsolve Test ---

# Dimension
dim = 4

# == Define Symbolic Structure (Full Sequence) ==
# Use the original couplings structure to define the sequence of variables
# The actual coupling values (indices) for symbolic U will come from the *fixed* list
original_couplings = [(0, 3), (0, 1), (0, 2), (0, 1), (0, 3)]
original_fixed_phase_flags = [1.5, 0.5, 1.5, 0.5, 0.5] # Example values, needed for fixing structure
num_pulses = len(original_couplings)

# Get the actual sequence structure AFTER fixing
sequence_struct, _ = fix_couplings_and_phases(
    original_couplings, original_fixed_phase_flags
)
print(f"Symbolic sequence structure (couplings): {sequence_struct}")
if len(sequence_struct) != num_pulses:
     warnings.warn("Mismatch in length after fixing - check logic.")


# Define symbolic parameters (theta_0..4, phi_0..4)
params = []
for k in range(num_pulses):
    theta = sp.symbols(f'theta_{k}', real=True) # Represents pi * fraction_k
    phi = sp.symbols(f'phi_{k}', real=True)     # Represents pi * fixed_phase_flag_k
    params.extend([theta, phi])

print(f"\nSymbolic parameters to solve for ({len(params)}): {params}")

# Build the full symbolic unitary U = G_N * ... * G_0
print("\nBuilding full symbolic unitary (this might take a moment)...")
start_build_time = time.time()
U_symbolic = sp.eye(dim)
for k, (i, j) in enumerate(sequence_struct):
    theta_k = params[2*k]
    phi_k = params[2*k+1]
    Gk = givens_rotation_symbolic(i, j, theta_k, phi_k, dim)
    U_symbolic = Gk * U_symbolic # Apply k-th pulse from the left
end_build_time = time.time()
print(f"Symbolic unitary built in {end_build_time - start_build_time:.2f} seconds.")
# print("\nSymbolic Unitary (Structure - showing only U[0,0]):") # Printing the whole thing is too much
# sp.pprint(U_symbolic[0,0])


# --- Generate the Target Unitary (Full Sequence) ---
target_couplings = [(0, 3), (0, 1), (0, 2), (0, 1), (0, 3)]
target_fractions = np.array([0.5, 2.0 * np.arcsin(np.sqrt(1/3))/np.pi, 4/3, 2.0 * np.arcsin(np.sqrt(2/3))/np.pi + 1, 0.5])
target_fixed_phase_flags = np.array([1.5, 0.5, 1.5, 0.5, 0.5])
target_rabi_freqs = np.array([1,1,1,1,1]) # Needed for 'unitary' function

# Apply fixing *before* generating target
target_couplings_fixed, target_fixed_phase_flags_fixed = fix_couplings_and_phases(
    target_couplings, target_fixed_phase_flags
)
# Ensure the structure used for target matches the symbolic sequence
if target_couplings_fixed != sequence_struct:
    warnings.warn("Mismatch between target generation couplings and symbolic sequence structure!")
    print("Target uses:", target_couplings_fixed)
    print("Symbolic uses:", sequence_struct)


# Calculate expected parameters (for verification and initial guess)
expected_thetas = target_fractions * np.pi
expected_phis = target_fixed_phase_flags * np.pi
expected_params = []
print("\nExpected Parameters (Numeric):")
for k in range(num_pulses):
    print(f"  theta_{k}: {expected_thetas[k]:.4f}, phi_{k}: {expected_phis[k]:.4f}")
    expected_params.extend([expected_thetas[k], expected_phis[k]])


# Generate the target U using the original method with the full sequence
target_U = unitary(target_couplings_fixed,
                   target_rabi_freqs,
                   target_fractions,
                   target_fixed_phase_flags_fixed,
                   dim)

print("\nTarget Unitary (Numeric, Rounded):")
print(np.round(target_U, 3))
target_U_sym = sp.Matrix(target_U) # Convert to sympy Matrix for equations

# --- Set up Equations for nsolve (N equations for N unknowns) ---
# Need 10 equations for 10 unknowns. Let's use Re/Im parts of:
# U[0,0], U[0,1], U[0,2], U[0,3], U[1,0]
equations = []
elements_to_use =[(0,0), (1,1), (2,2), (3,3),(0,1)]

print("\nSelecting equations from elements:", elements_to_use)
for i, j in elements_to_use:
    diff = U_symbolic[i, j] - target_U_sym[i, j]
    equations.append(sp.re(diff))
    equations.append(sp.im(diff))

print(f"Generated {len(equations)} equations for {len(params)} unknowns.")
if len(equations) != len(params):
     raise ValueError("Number of equations does not match number of parameters!")

# --- Define Initial Guesses ---
# Use values slightly perturbed from the expected ones
initial_guesses = [p * (1 + 0.1 * (np.random.rand() - 0.5)) for p in expected_params] # Perturb by up to +/- 5%
print(f"\nInitial Guesses (Perturbed): {np.round(initial_guesses, 4)}")

# --- Solve Numerically using nsolve ---
print("\nAttempting to solve numerically using nsolve...")
print("WARNING: This might take a significant amount of time!")
start_solve_time = time.time()
try:
    # nsolve requires equations, variables, and initial guesses
    # May need to adjust solver options like 'tol' or 'maxsteps' if it struggles
    # solutions_nsolve = sp.nsolve(equations, params, initial_guesses, prec=30) # Increase precision if needed
    solutions_nsolve = sp.nsolve(equations, params, initial_guesses)


    end_solve_time = time.time()
    print(f"nsolve finished in {end_solve_time - start_solve_time:.2f} seconds.")

    print("\nNumerical Solution Found:")
    # nsolve returns a Matrix object
    solved_params = solutions_nsolve.transpose().tolist()[0] # Extract solutions as a list

    print("\nComparison with Expected Values:")
    param_names = [str(p) for p in params]
    max_abs_error = 0
    max_mod_error = 0
    for k in range(len(params)):
        solved = solved_params[k]
        expected = expected_params[k]
        abs_error = abs(solved - expected)
        max_abs_error = max(max_abs_error, abs_error)
        print(f"  {param_names[k]:<8}: Solved={solved:<8.4f} Expected={expected:<8.4f} AbsError={abs_error:<.2e}", end="")

        # Check phase errors modulo 2*pi
        if 'phi' in param_names[k]:
             mod_error = abs( (solved - expected + np.pi) % (2*np.pi) - np.pi )
             max_mod_error = max(max_mod_error, mod_error)
             print(f" ModError={mod_error:<.2e}")
        else:
             print("") # newline for theta

    print(f"\nMax Absolute Parameter Error: {max_abs_error:.2e}")
    print(f"Max Phase Error (mod 2pi): {max_mod_error:.2e}")


    # --- Verification ---
    print("\nVerifying solution:")
    sub_dict = {p: val for p, val in zip(params, solved_params)}
    # Evaluating symbolic U with solved params can also be slow
    print("Evaluating symbolic U with solved parameters...")
    start_verify_time = time.time()
    U_check_sym = U_symbolic.subs(sub_dict)
    end_verify_time = time.time()
    print(f"Evaluation took {end_verify_time - start_verify_time:.2f} seconds.")

    # Convert symbolic matrix with numbers to numpy array
    try:
        U_check_np = np.array(U_check_sym.tolist()).astype(complex)
        print("\nSymbolic U evaluated with nsolve solution (Rounded):")
        print(np.round(U_check_np, 3))

        print("\nDifference between Target U and Calculated U (Magnitude):")
        diff_mag = np.abs(target_U - U_check_np)
        print(np.round(diff_mag, 5))
        max_diff = np.max(diff_mag)
        print(f"Max element-wise difference: {max_diff:.2e}")
        print(f"Matrices are close (tol=1e-5): {np.allclose(target_U, U_check_np, atol=1e-5)}") # Use tolerance

    except Exception as e:
        print(f"\nCould not convert solution matrix to numpy: {e}")
        print("This might happen if the solution contains symbolic constants (like pi).")


except (NotImplementedError, TypeError) as e:
    print(f"\nSolver failed: {e}")
    print("The symbolic expressions might be too complex, or there might be type issues.")
except ValueError as e:
     print(f"\nSolver failed: {e}")
     print("This often happens if the solver cannot find a solution from the initial guess.")
     print("Try different initial guesses or equation selections.")
except Exception as e:
    print(f"\nAn unexpected error occurred during solving: {e}")

print("\n--- Script Finished ---")

Symbolic sequence structure (couplings): [(0, 3), (0, 1), (0, 2), (0, 1), (0, 3)]

Symbolic parameters to solve for (10): [theta_0, phi_0, theta_1, phi_1, theta_2, phi_2, theta_3, phi_3, theta_4, phi_4]

Building full symbolic unitary (this might take a moment)...
Symbolic unitary built in 0.00 seconds.

Expected Parameters (Numeric):
  theta_0: 1.5708, phi_0: 4.7124
  theta_1: 1.2310, phi_1: 1.5708
  theta_2: 4.1888, phi_2: 4.7124
  theta_3: 5.0522, phi_3: 1.5708
  theta_4: 1.5708, phi_4: 1.5708

Target Unitary (Numeric, Rounded):
[[ 0.5+0.j  0.5-0.j  0.5-0.j  0.5-0.j]
 [ 0.5+0.j -0.5-0.j  0.5-0.j -0.5+0.j]
 [ 0.5+0.j  0.5+0.j -0.5-0.j -0.5-0.j]
 [ 0.5+0.j -0.5-0.j -0.5+0.j  0.5+0.j]]

Selecting equations from elements: [(0, 0), (1, 1), (2, 2), (3, 3), (0, 1)]
Generated 10 equations for 10 unknowns.

Initial Guesses (Perturbed): [1.4946 4.8244 1.19   1.5556 4.1148 4.8195 5.0862 1.5839 1.5313 1.5854]

Attempting to solve numerically using nsolve...

An unexpected error occurred during 

In [114]:
import sympy as sp
from scipy.linalg import expm
import numpy as np
import matplotlib.pyplot as plt
import warnings
import time

def coupling_operator_with_phase(i, j, dim, phi):
    op = np.zeros((dim, dim), dtype=complex)
    op[i, j] = np.exp(+1j * phi)
    op[j, i] = np.exp(-1j * phi)
    return op

def pulse_duration_for_fraction(f, Omega):
    theta = np.pi*np.array(f)
    return theta / Omega if Omega != 0 else 0.0

def unitary(couplings, rabi_freqs, fractions, fixed_phase_flags, dim):
    U_seq = np.eye(dim, dtype=complex)
    for (levels, Omega, frac, fix_pflag) in zip(couplings, rabi_freqs, fractions, fixed_phase_flags):
        i, j = levels
        phi_fixed = fix_pflag * np.pi
        total_phase = phi_fixed
        H_op = coupling_operator_with_phase(i, j, dim, total_phase)
        H_coupling = 0.5 * Omega * H_op
        t_pulse = pulse_duration_for_fraction(frac, Omega)
        U_pulse = expm(-1j * H_coupling * t_pulse)
        U_seq = U_pulse @ U_seq
    return U_seq

def fix_couplings_and_phases(couplings, fixed_phase_flags):
    new_couplings = []
    new_fixed_phase_flags = []
    for (cpl, phase_flag) in zip(couplings, fixed_phase_flags):
        i, j = cpl
        if i != 0 and j == 0:
            cpl_fixed = (0, i)
            phase_flag_fixed = phase_flag + 1.0
        else:
            cpl_fixed = cpl
            phase_flag_fixed = phase_flag
        new_couplings.append(cpl_fixed)
        new_fixed_phase_flags.append(phase_flag_fixed)
    return new_couplings, new_fixed_phase_flags

def givens_rotation_symbolic(i, j, theta, phi, dim):
    G = sp.eye(dim)
    cos_term = sp.cos(theta / 2)
    sin_term = sp.sin(theta / 2)
    e_neg_iphi = sp.exp(-sp.I * phi)
    e_pos_iphi = sp.conjugate(e_neg_iphi)

    G[i, i] = cos_term
    G[j, j] = cos_term
    G[i, j] = -sp.I * e_pos_iphi * sin_term
    G[j, i] = -sp.I * e_neg_iphi * sin_term
    return G


dim = 8

original_couplings = [(0, 3), (0, 1), (0, 2), (0, 1), (0, 3)]
original_fixed_phase_flags = [1.5, 0.5, 1.5, 0.5, 0.5] 



# sequence_struct, _ = fix_couplings_and_phases(
#     original_couplings, original_fixed_phase_flags
# )

sequence_struct = [(0, 7), (0, 6), (0, 5), (0, 4), (0, 3), (0, 2), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7)]
num_pulses = len(sequence_struct)
print(f"Symbolic sequence structure (couplings): {sequence_struct}")

# Define symbolic parameters (theta_0..4, phi_0..4)
params = []
for k in range(num_pulses):
    theta = sp.symbols(f'theta_{k}', real=True)
    phi = sp.symbols(f'phi_{k}', real=True)
    params.extend([theta, phi])
print(f"\nSymbolic parameters ({len(params)}): {params}")


start_build_time = time.time()
U_symbolic = sp.eye(dim)
for k, (i, j) in enumerate(sequence_struct):
    theta_k = params[2*k]
    phi_k = params[2*k+1]
    Gk = givens_rotation_symbolic(i, j, theta_k, phi_k, dim)
    U_symbolic = Gk * U_symbolic
end_build_time = time.time()
print(f"Symbolic unitary built in {end_build_time - start_build_time:.2f} seconds.")


# target_couplings = [(0, 3), (0, 1), (0, 2), (0, 1), (0, 3)]
# target_fractions = np.array([0.5, 2.0 * np.arcsin(np.sqrt(1/3))/np.pi, 4/3, 2.0 * np.arcsin(np.sqrt(2/3))/np.pi + 1, 0.5])
# target_fixed_phase_flags = np.array([1.5, 0.5, 1.5, 0.5, 0.5])
# target_rabi_freqs = np.array([1,1,1,1,1])

# target_couplings_fixed, target_fixed_phase_flags_fixed = fix_couplings_and_phases(
#     target_couplings, target_fixed_phase_flags
# )


# target_U = np.round(unitary(target_couplings_fixed,
#                    target_rabi_freqs,
#                    target_fractions,
#                    target_fixed_phase_flags_fixed,
#                    dim),3)

target_U = np.array([[0.354-0.j,  0.354+0.j,  0.354+0.j,  0.354+0.j,  0.354+0.j,
   0.354+0.j,  0.354+0.j,  0.354+0.j],
 [0.354-0.j, -0.354+0.j,  0.354+0.j, -0.354-0.j,  0.354+0.j,
  -0.354-0.j,  0.354+0.j, -0.354-0.j],
 [0.354-0.j,  0.354-0.j, -0.354-0.j, -0.354+0.j,  0.354+0.j,
   0.354+0.j, -0.354-0.j, -0.354-0.j],
 [0.354-0.j, -0.354+0.j, -0.354+0.j,  0.354-0.j,  0.354+0.j,
  -0.354-0.j, -0.354-0.j,  0.354+0.j],
 [0.354-0.j,  0.354-0.j,  0.354-0.j,  0.354-0.j, -0.354-0.j,
  -0.354-0.j, -0.354-0.j, -0.354-0.j],
 [0.354-0.j, -0.354+0.j,  0.354-0.j, -0.354+0.j, -0.354+0.j,
   0.354+0.j, -0.354-0.j,  0.354+0.j],
 [0.354-0.j, 0.354-0.j, -0.354+0.j, -0.354+0.j, -0.354+0.j,
  -0.354+0.j,  0.354+0.j, 0.354+0.j],
 [0.354-0.j, -0.354+0.j, -0.354+0.j,  0.354-0.j, -0.354+0.j,
   0.354+0.j,  0.354+0.j, -0.354+0.j]],  dtype=complex)
print("\nTarget Unitary:")
print(np.round(target_U, 3))

target_U_sym = sp.Matrix(target_U)


all_equations = []
equation_labels = []

for i in range(dim):
    for j in range(dim):
        # Define the difference for element (i, j)
        print(target_U_sym[i, j])
        diff = U_symbolic[i, j] - target_U_sym[i, j]
        # Add the real part = 0 equation
        all_equations.append(sp.re(diff))
        equation_labels.append(f"Re(Eq[{i},{j}])")
        # Add the imaginary part = 0 equation
        all_equations.append(sp.im(diff))
        equation_labels.append(f"Im(Eq[{i},{j}])")

print(f"\n--- Generated System of {len(all_equations)} Equations ---")
print(f"--- Variables: {params} ---")


for k, (label, eq) in enumerate(zip(equation_labels, all_equations)):
    print(f"\n--- Equation {k+1}: {label} ---")

    # sp.pprint(eq, wrap_line=False,use_unicode=True)
    print(eq)
    print("= 0")



Symbolic sequence structure (couplings): [(0, 7), (0, 6), (0, 5), (0, 4), (0, 3), (0, 2), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7)]

Symbolic parameters (26): [theta_0, phi_0, theta_1, phi_1, theta_2, phi_2, theta_3, phi_3, theta_4, phi_4, theta_5, phi_5, theta_6, phi_6, theta_7, phi_7, theta_8, phi_8, theta_9, phi_9, theta_10, phi_10, theta_11, phi_11, theta_12, phi_12]
Symbolic unitary built in 0.14 seconds.

Target Unitary:
[[ 0.354+0.j  0.354+0.j  0.354+0.j  0.354+0.j  0.354+0.j  0.354+0.j
   0.354+0.j  0.354+0.j]
 [ 0.354+0.j -0.354+0.j  0.354+0.j -0.354+0.j  0.354+0.j -0.354+0.j
   0.354+0.j -0.354+0.j]
 [ 0.354+0.j  0.354+0.j -0.354+0.j -0.354+0.j  0.354+0.j  0.354+0.j
  -0.354+0.j -0.354+0.j]
 [ 0.354+0.j -0.354+0.j -0.354+0.j  0.354+0.j  0.354+0.j -0.354+0.j
  -0.354+0.j  0.354+0.j]
 [ 0.354+0.j  0.354+0.j  0.354+0.j  0.354+0.j -0.354+0.j -0.354+0.j
  -0.354+0.j -0.354+0.j]
 [ 0.354+0.j -0.354+0.j  0.354+0.j -0.354+0.j -0.354+0.j  0.354+0.j
  -0.354+0.j  0.354+0.

In [102]:
import numpy as np
from scipy.linalg import expm
from scipy.optimize import least_squares
import matplotlib.pyplot as plt
import time
import itertools

# --- Helper functions ---

# Keep unitary() and fix_couplings_and_phases() exactly as before
# to generate the target matrix accurately.
def coupling_operator_with_phase(i, j, dim, phi):
    op = np.zeros((dim, dim), dtype=complex)
    op[i, j] = np.exp(+1j * phi)
    op[j, i] = np.exp(-1j * phi)
    return op

def pulse_duration_for_fraction(f, Omega):
    theta = np.pi*np.array(f)
    return theta / Omega if Omega != 0 else 0.0

def unitary(couplings, rabi_freqs, fractions, fixed_phase_flags, dim):
    U_seq = np.eye(dim, dtype=complex)
    for (levels, Omega, frac, fix_pflag) in zip(couplings, rabi_freqs, fractions, fixed_phase_flags):
        i, j = levels
        phi_fixed = fix_pflag * np.pi
        total_phase = phi_fixed
        H_op = coupling_operator_with_phase(i, j, dim, total_phase)
        H_coupling = 0.5 * Omega * H_op
        t_pulse = pulse_duration_for_fraction(frac, Omega)
        U_pulse = expm(-1j * H_coupling * t_pulse)
        U_seq = U_pulse @ U_seq
    return U_seq

def fix_couplings_and_phases(couplings, fixed_phase_flags):
    new_couplings = []
    new_fixed_phase_flags = []
    for (cpl, phase_flag) in zip(couplings, fixed_phase_flags):
        i, j = cpl
        if i != 0 and j == 0:
            cpl_fixed = (0, i)
            phase_flag_fixed = phase_flag + 1.0
        else:
            cpl_fixed = cpl
            phase_flag_fixed = phase_flag
        new_couplings.append(cpl_fixed)
        new_fixed_phase_flags.append(phase_flag_fixed)
    return new_couplings, new_fixed_phase_flags

# --- Numerical Unitary Calculation ---

def givens_rotation_numerical(i, j, theta, phi, dim):
    """
    Calculates NUMERICAL complex Givens rotation matrix.
    theta: Represents the pulse angle (pi * fraction).
    phi: Represents the pulse phase (phase_flag * pi).
    """
    G = np.eye(dim, dtype=complex)
    cos_term = np.cos(np.pi*theta / 2.0)
    sin_term = np.sin(np.pi*theta / 2.0)
    # Use numpy's complex exponential
    e_neg_iphi = np.exp(-1j * phi*np.pi)
    e_pos_iphi = np.conjugate(e_neg_iphi) # Or np.exp(1j * phi)

    G[i, i] = cos_term
    G[j, j] = cos_term
    G[i, j] = -1j * e_pos_iphi * sin_term
    G[j, i] = -1j * e_neg_iphi * sin_term
    return G

def calculate_unitary_numerical(params, sequence_struct, dim):
    """
    Calculates the final unitary numerically from parameters.
    params: 1D array [theta_0, phi_0, theta_1, phi_1, ...]
    sequence_struct: List of tuples (i, j) for couplings.
    dim: Dimension of the matrix.
    """
    U_calc = np.eye(dim, dtype=complex)
    num_pulses = len(sequence_struct)
    if len(params) != 2 * num_pulses:
        raise ValueError("Length of params must be 2 * number of pulses")

    for k, (i, j) in enumerate(sequence_struct):
        theta_k = params[2*k]
        phi_k = params[2*k+1]
        Gk = givens_rotation_numerical(i, j, theta_k, phi_k, dim)
        U_calc = Gk @ U_calc # Apply k-th pulse from the left: U = G_N...G_1
    return U_calc

# --- Residuals Function for Least Squares ---

def residuals(params, target_U, sequence_struct, dim):
    """
    Calculates the residuals vector for least_squares.
    Residuals = [Re(U_calc - U_target), Im(U_calc - U_target)] flattened.
    """
    # Calculate the unitary matrix for the current params
    U_calc = calculate_unitary_numerical(params, sequence_struct, dim)

    # Calculate the difference matrix
    delta_U = U_calc - target_U

    # Flatten the difference matrix and separate real/imaginary parts
    # Method 1: Ravel and stack
    # residuals_vec = np.concatenate((np.real(delta_U).ravel(), np.imag(delta_U).ravel()))

    # Method 2: Interleave Re/Im parts (often numerically equivalent)
    residuals_vec = np.zeros(2 * dim * dim)
    residuals_vec[0::2] = np.real(delta_U).ravel() # Real parts at even indices
    residuals_vec[1::2] = np.imag(delta_U).ravel() # Imaginary parts at odd indices

    return residuals_vec

# --- Main Script ---

# Dimension
dim = 8

# --- Define Target Sequence ---
# target_couplings = [(0, 3), (0, 1), (0, 2), (0, 1), (0, 3)]
# target_fractions = np.array([0.5, 2.0 * np.arcsin(np.sqrt(1/3))/np.pi, 4/3, 2.0 * np.arcsin(np.sqrt(2/3))/np.pi + 1, 0.5])
# target_fixed_phase_flags = np.array([1.5, 0.5, 1.5, 0.5, 0.5])
target_couplings = [(0, 2), (0, 5), (0, 3), (0, 8), (0, 1), (0, 4), (0, 6), (0, 7), (0, 6), (0, 4), (0, 1), (0, 8), (0, 3), (0, 5), (0, 2)]
target_fractions = np.array([1.0, 0.5000000000000001, 0.3918265520306073, 0.33333333333333337, 0.2951672353008665, 0.26772047280123, 0.24675171442884983, 0.2300534561626159, 0.24675171442884983, 0.26772047280123, 0.2951672353008665, 0.33333333333333337, 0.3918265520306073, 0.5000000000000001, 1.0])
target_fixed_phase_flags = np.array([0.5]*len(target_couplings))

target_rabi_freqs = np.array([1]*len(target_couplings)) # Needed ONLY for target generation

# Apply fixing *before* generating target AND defining sequence structure
# target_couplings_fixed, target_fixed_phase_flags_fixed = fix_couplings_and_phases(
#     target_couplings, target_fixed_phase_flags
# )
l_list = [(0,7),(0,2),(0,3),(0,4),(0,5),(0,6)]
center = [(0,1)]
permutations = list(itertools.permutations(l_list))

# target_U = unitary(target_couplings_fixed,
#                    target_rabi_freqs,
#                    target_fractions,
#                    target_fixed_phase_flags_fixed,
#                    dim)

U = np.array([[0.354-0.j,  0.354+0.j,  0.354+0.j,  0.354+0.j,  0.354+0.j,
   0.354+0.j,  0.354+0.j,  0.354+0.j],
 [0.354-0.j, -0.354+0.j,  0.354+0.j, -0.354-0.j,  0.354+0.j,
  -0.354-0.j,  0.354+0.j, -0.354-0.j],
 [0.354-0.j,  0.354-0.j, -0.354-0.j, -0.354+0.j,  0.354+0.j,
   0.354+0.j, -0.354-0.j, -0.354-0.j],
 [0.354-0.j, -0.354+0.j, -0.354+0.j,  0.354-0.j,  0.354+0.j,
  -0.354-0.j, -0.354-0.j,  0.354+0.j],
 [0.354-0.j,  0.354-0.j,  0.354-0.j,  0.354-0.j, -0.354-0.j,
  -0.354-0.j, -0.354-0.j, -0.354-0.j],
 [0.354-0.j, -0.354+0.j,  0.354-0.j, -0.354+0.j, -0.354+0.j,
   0.354+0.j, -0.354-0.j,  0.354+0.j],
 [0.354-0.j, 0.354-0.j, -0.354+0.j, -0.354+0.j, -0.354+0.j,
  -0.354+0.j,  0.354+0.j, 0.354+0.j],
 [0.354-0.j, -0.354+0.j, -0.354+0.j,  0.354-0.j, -0.354+0.j,
   0.354+0.j,  0.354+0.j, -0.354+0.j]],  dtype=complex)

target_U = U

print(np.round(target_U, 3))

for l in permutations:
    l_l = list(l)
    # print(center)
    sequence_struct = l_l + center + l_l[::-1]
    num_pulses = len(sequence_struct)
    num_params = 2 * num_pulses
    # print(f"Sequence structure (fixed couplings): {sequence_struct}")
    
    # --- Generate the Target Unitary (Once) ---
    # print("\nGenerating target unitary...")

    
    # --- Calculate Expected Parameters and Initial Guess ---
    expected_thetas = target_fractions 
    expected_phis = target_fixed_phase_flags 
    expected_params = np.zeros(num_params)
    # print("\nExpected Parameters (Numeric):")
    for k in range(num_pulses):
        expected_params[2*k] = expected_thetas[k]
        expected_params[2*k+1] = expected_phis[k]
        # print(f"  theta_{k}: {expected_thetas[k]:.4f}, phi_{k}: {expected_phis[k]:.4f}")
    
    # Define Initial Guess x0 (e.g., perturb expected values)
    np.random.seed(42) # for reproducibility
    initial_guess = [0.5]*len(expected_params) 
    # print(f"\nInitial Guess (Perturbed): {np.round(initial_guess, 4)}")
    
    # --- Perform Least Squares Optimization ---
    # print("\nStarting least_squares optimization...")
    # print(f"Minimizing sum of squares of {2*dim*dim} residuals for {num_params} parameters.")
    start_time = time.time()
    
    # Call least_squares
    # Args are passed to the residuals function after the parameters vector
    # verbose=2 shows detailed iteration progress
    # print(initial_guess)
    # lower_bounds = [0.0] * len(initial_guess)
    # upper_bounds = [2.0] * len(initial_guess)
    # bounds = (lower_bounds, upper_bounds)
    
    # print(bounds)
    result = least_squares(
        residuals,
        initial_guess,
        # bounds=bounds,
        args=(target_U, sequence_struct, dim),
        jac='3-point', # More robust Jacobian estimation than '2-point'
        method='trf', # 'trf' or 'lm'. 'trf' can handle bounds if needed later.
        verbose=0,
        ftol=1e-8, # Tighter tolerances might be needed
        xtol=1e-8,
        gtol=1e-8
    )
    
    end_time = time.time()
    print(f"\nOptimization finished in {end_time - start_time:.2f} seconds.")
    
    # # --- Analyze Results ---
    # print("\n--- Optimization Results ---")
    # print(f"Success: {result.success}")
    # print(f"Status: {result.status} ({result.message})")
    # # Cost is 0.5 * sum(residuals**2). Should be very close to 0 for a good fit.
    # print(f"Final Cost (0.5 * sum(residuals^2)): {result.cost:.3e}")
    
    # Extract the found parameters
    solved_params = result.x
    U_check_np = calculate_unitary_numerical(solved_params, sequence_struct, dim)
    if np.allclose(target_U, U_check_np, atol=1e-6):
        
        print("\nFound Parameters vs Expected:")
        max_abs_error = 0
        max_mod_error = 0
        param_names = [f'theta_{k}' if i % 2 == 0 else f'phi_{k//2}' for i, k in enumerate([i//2 for i in range(num_params)])]
        
        for k in range(num_params):
            solved = solved_params[k]
            expected = expected_params[k]
            abs_error = abs(solved - expected)
            max_abs_error = max(max_abs_error, abs_error)
            print(f"  {param_names[k]:<8}: Solved={solved:<8.4f} Expected={expected:<8.4f} AbsError={abs_error:<.2e}", end="")
        
            # Check phase errors modulo 2*pi
            if 'phi' in param_names[k]:
                 # Normalize expected phase to [0, 2pi) or (-pi, pi] if desired before comparison
                 # expected_norm = expected % (2 * np.pi)
                 # solved_norm = solved % (2 * np.pi)
                 # mod_error = abs( (solved_norm - expected_norm + np.pi) % (2*np.pi) - np.pi )
                 mod_error = abs( (solved - expected + np.pi) % (2*np.pi) - np.pi ) # More direct comparison
                 max_mod_error = max(max_mod_error, mod_error)
                 print(f" ModError={mod_error:<.2e}")
            else:
                 print("") # newline for theta
        
        print(f"\nMax Absolute Parameter Error: {max_abs_error:.2e}")
        print(f"Max Phase Error (mod 2pi): {max_mod_error:.2e}")
        
        # --- Verification ---
        print("\nVerifying solution by recalculating Unitary:")
    
        
        print("\nTarget U (Rounded):")
        print(np.round(target_U, 3))
        print("\nCalculated U with found parameters (Rounded):")
        print(np.round(U_check_np, 3))
        
        print("\nDifference between Target U and Calculated U (Magnitude):")
        diff_mag = np.abs(target_U - U_check_np)
        print(np.round(diff_mag, 5))
        max_diff = np.max(diff_mag)
        frob_norm_diff = np.linalg.norm(target_U - U_check_np, 'fro')
        print(f"\nMax element-wise difference: {max_diff:.2e}")
        print(f"Frobenius norm of difference: {frob_norm_diff:.2e}")
        # Check if cost relates to Frobenius norm: cost = 0.5 * ||residuals||^2 = 0.5 * ||U_calc - U_target||_F^2
        print(f"Check: 0.5 * (Frob Norm)^2 = {0.5 * frob_norm_diff**2:.3e} (should approx match cost)")
        print(f"Matrices are close (atol=1e-6): {np.allclose(target_U, U_check_np, atol=1e-6)}") 
        
print("\n--- Script Finished ---")

[[ 0.354+0.j  0.354+0.j  0.354+0.j  0.354+0.j  0.354+0.j  0.354+0.j
   0.354+0.j  0.354+0.j]
 [ 0.354+0.j -0.354+0.j  0.354+0.j -0.354+0.j  0.354+0.j -0.354+0.j
   0.354+0.j -0.354+0.j]
 [ 0.354+0.j  0.354+0.j -0.354+0.j -0.354+0.j  0.354+0.j  0.354+0.j
  -0.354+0.j -0.354+0.j]
 [ 0.354+0.j -0.354+0.j -0.354+0.j  0.354+0.j  0.354+0.j -0.354+0.j
  -0.354+0.j  0.354+0.j]
 [ 0.354+0.j  0.354+0.j  0.354+0.j  0.354+0.j -0.354+0.j -0.354+0.j
  -0.354+0.j -0.354+0.j]
 [ 0.354+0.j -0.354+0.j  0.354+0.j -0.354+0.j -0.354+0.j  0.354+0.j
  -0.354+0.j  0.354+0.j]
 [ 0.354+0.j  0.354+0.j -0.354+0.j -0.354+0.j -0.354+0.j -0.354+0.j
   0.354+0.j  0.354+0.j]
 [ 0.354+0.j -0.354+0.j -0.354+0.j  0.354+0.j -0.354+0.j  0.354+0.j
   0.354+0.j -0.354+0.j]]

Optimization finished in 0.82 seconds.

Optimization finished in 0.44 seconds.

Optimization finished in 0.88 seconds.

Optimization finished in 0.96 seconds.

Optimization finished in 1.02 seconds.

Optimization finished in 0.74 seconds.

Optimization f